In [ ]:
# -*- coding: utf-8 -*-
"""
v19_part1_feature_engineering.py (Nihai Kapsamlı Mimari - Özellik Mühendisliği)

Bu script, projenin tüm özellik mühendisliği adımlarını içerir:
1.  Temel veri hazırlığı ve Özgün Etki Skorlarının hesaplanması.
2.  Kullanıcı ve kategori için meta-modellerin eğitilmesi. (ÜRÜN MODELİ KALDIRILDI)
3.  Tüm v7 özellikleri, v16 koşullu sızıntı özellikleri ve v18 meta-özelliklerini
    birleştiren nihai, kapsamlı özellik setinin oluşturulması ve diske kaydedilmesi.
"""


import pandas as pd
import numpy as np
import os
import gc
from catboost import CatBoostRegressor
import json
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

# ==============================================================================
# --- BÖLÜM 1: TEMEL VERİ HAZIRLIĞI VE SKORLARIN HESAPLANMASI (v19) ---
# ==============================================================================
print("="*50)
print("Bölüm 1: Temel Veri ve Skorların Hesaplanması (v19) Başladı.")
print("="*50)

# --- Dosya Yolları ---
BASE_DIR = './'
os.makedirs(os.path.join(BASE_DIR, 'processed'), exist_ok=True)
os.makedirs(os.path.join(BASE_DIR, 'models'), exist_ok=True)
os.makedirs(os.path.join(BASE_DIR, 'submissions'), exist_ok=True)


OUT_TRAIN_PATH = os.path.join(BASE_DIR, 'processed/train_processed_v19.csv')
OUT_TEST_PATH = os.path.join(BASE_DIR, 'processed/test_processed_v19.csv')

IN_TRAIN_PATH = os.path.join(BASE_DIR, 'train.csv')
IN_TEST_PATH = os.path.join(BASE_DIR, 'test.csv')

# --- Veri Yükleme ve Ön İşleme ---
print("Ham veriler yükleniyor ve ön işleniyor...")
df_train_raw = pd.read_csv(IN_TRAIN_PATH, parse_dates=['event_time'])
df_test_raw = pd.read_csv(IN_TEST_PATH, parse_dates=['event_time'])

def fix_anomalous_sessions(df):
    session_user_counts = df.groupby('user_session')['user_id'].nunique()
    anomalous_sessions = session_user_counts[session_user_counts > 1].index
    if len(anomalous_sessions) > 0:
        anomalous_indices = df['user_session'].isin(anomalous_sessions)
        df['user_session_original'] = df['user_session']
        df.loc[anomalous_indices, 'user_session'] = df.loc[anomalous_indices, 'user_session'] + '_' + df.loc[anomalous_indices, 'user_id'].astype(str)
    else:
        df['user_session_original'] = df['user_session']
    return df

df_train_raw = fix_anomalous_sessions(df_train_raw)
df_test_raw = fix_anomalous_sessions(df_test_raw)

df_combined = pd.concat([df_train_raw.drop('session_value', axis=1), df_test_raw], ignore_index=True)

# --- Ürün Filtreleme ---
print("Ürün sayıları hesaplanıyor ve filtreleme yapılıyor...")
product_counts = df_train_raw['product_id'].value_counts()
print(f"Toplam ürün sayısı: {len(product_counts)}")

# 10'dan az kez geçen ürünleri bulma
products_less_than_10 = product_counts[product_counts < 10]
print(f"10'dan az kez geçen ürün sayısı: {len(products_less_than_10)}")

# Bu ürünleri train verisinden çıkarma
df_train_filtered = df_train_raw[~df_train_raw['product_id'].isin(products_less_than_10.index)].copy()
print(f"Filtreleme öncesi train boyutu: {len(df_train_raw)}")
print(f"Filtreleme sonrası train boyutu: {len(df_train_filtered)}")

# Test verisini de aynı şekilde filtrele
df_test_filtered = df_test_raw[~df_test_raw['product_id'].isin(products_less_than_10.index)].copy()
print(f"Filtreleme sonrası test boyutu: {len(df_test_filtered)}")

# Filtrelenmiş veriler ile devam et
df_combined = pd.concat([df_train_filtered.drop('session_value', axis=1), df_test_filtered], ignore_index=True)

# --- Özgün Etki Skorları için Baseline ve Delta Hesaplaması ---
print("Özgün Etki Skorları için baseline'lar ve delta'lar hesaplanıyor...")
event_map = {'VIEW': 'V', 'ADD_CART': 'A', 'REMOVE_CART': 'R', 'BUY': 'B'}
df_combined['event_short'] = df_combined['event_type'].map(event_map)
signatures = df_combined.groupby('user_session')['event_short'].apply(lambda x: '_'.join(sorted(x)))
df_train_filtered = df_train_filtered.merge(signatures.rename('behavioral_signature'), on='user_session', how='left')
avg_value_for_signature = df_train_filtered.groupby('behavioral_signature')['session_value'].mean().to_dict()
df_train_filtered['baseline_value'] = df_train_filtered['behavioral_signature'].map(avg_value_for_signature)
df_train_filtered['delta'] = df_train_filtered['session_value'] - df_train_filtered['baseline_value']

user_impact_scores = df_train_filtered.groupby('user_id')['delta'].mean().reset_index().rename(columns={'delta': 'actual_user_impact'})
category_impact_scores = df_train_filtered.groupby('category_id')['delta'].mean().reset_index().rename(columns={'delta': 'actual_cat_impact'})

# Ürün etki skoru hesaplaması geri eklendi
product_impact_scores = df_train_filtered.groupby('product_id')['delta'].mean().reset_index().rename(columns={'delta': 'actual_prod_impact'})

# ==============================================================================
# --- BÖLÜM 2: META-MODELLERİN EĞİTİMİ (v19) ---
# ==============================================================================
print("\n" + "="*50)
print("Bölüm 2: Meta-Modellerin Eğitimi (v19) Başladı.")
print("="*50)

# --- 2.1 Kullanıcı Meta-Modeli ---
print("Kullanıcı meta-modeli eğitiliyor...")
user_agg = df_combined.groupby('user_id').agg(
    user_event_count=('event_type', 'size'),
    user_session_count=('user_session', 'nunique'),
    user_nunique_products=('product_id', 'nunique'),
    user_nunique_categories=('category_id', 'nunique')
).reset_index()
user_meta_data = user_agg.merge(user_impact_scores, on='user_id', how='left')

user_meta_data_train = user_meta_data.dropna(subset=['actual_user_impact'])
X_user_meta, y_user_meta = user_meta_data_train.drop(['user_id', 'actual_user_impact'], axis=1), user_meta_data_train['actual_user_impact']
# DEĞİŞİKLİK: Parametreler artırıldı
user_meta_model = CatBoostRegressor(iterations=3000, depth=7, verbose=0, random_seed=42)
user_meta_model.fit(X_user_meta, y_user_meta)

# DEĞİŞİKLİK: Ürün meta-modeli bölümü geri eklendi.
# --- 2.2 Ürün Meta-Modeli ---
print("Ürün meta-modeli eğitiliyor...")
prod_agg = pd.crosstab(df_combined['product_id'], df_combined['event_type'])
prod_meta_data = prod_agg.merge(product_impact_scores, on='product_id', how='left')
prod_meta_data_train = prod_meta_data.dropna(subset=['actual_prod_impact'])
X_prod_meta, y_prod_meta = prod_meta_data_train.drop(['product_id', 'actual_prod_impact'], axis=1), prod_meta_data_train['actual_prod_impact']
prod_meta_model = CatBoostRegressor(iterations=3000, depth=7, verbose=0, random_seed=42)
prod_meta_model.fit(X_prod_meta, y_prod_meta)

# --- 2.3 Kategori Meta-Modeli ---
print("Kategori meta-modeli eğitiliyor...")
cat_agg = pd.crosstab(df_combined['category_id'], df_combined['event_type'])
cat_meta_data = cat_agg.merge(category_impact_scores, on='category_id', how='left')
cat_meta_data_train = cat_meta_data.dropna(subset=['actual_cat_impact'])
X_cat_meta, y_cat_meta = cat_meta_data_train.drop(['category_id', 'actual_cat_impact'], axis=1), cat_meta_data_train['actual_cat_impact']
# DEĞİŞİKLİK: Parametreler artırıldı
cat_meta_model = CatBoostRegressor(iterations=3000, depth=7, verbose=0, random_seed=42)
cat_meta_model.fit(X_cat_meta, y_cat_meta)

# ==============================================================================
# --- BÖLÜM 3: NİHAİ KAPSAMLI ÖZELLİK OLUŞTURMA (v19) ---
# ==============================================================================
print("\n" + "="*50)
print("Bölüm 3: Nihai Kapsamlı Özellik Oluşturma (v19) Başladı.")
print("="*50)

# --- 3.1 Meta-Tahminleri ve Değer Katma Skorlarını Hesaplama ---
print("Tüm elemanlar için meta-tahminler ve Değer Katma Skorları oluşturuluyor...")
all_users_agg = df_combined.groupby('user_id').agg(
    user_event_count=('event_type', 'size'), user_session_count=('user_session', 'nunique'),
    user_nunique_products=('product_id', 'nunique'), user_nunique_categories=('category_id', 'nunique')
).reset_index()
all_users_agg['predicted_user_impact'] = user_meta_model.predict(all_users_agg.drop('user_id', axis=1))
all_users_agg = all_users_agg.merge(user_impact_scores, on='user_id', how='left')
all_users_agg['user_value_add_score'] = all_users_agg['actual_user_impact'] - all_users_agg['predicted_user_impact']
user_final_scores = all_users_agg[['user_id', 'predicted_user_impact', 'user_value_add_score']]

# Ürün final skorları hesaplama bölümü geri eklendi.
all_prods_agg = pd.crosstab(df_combined['product_id'], df_combined['event_type'])
all_prods_agg['predicted_prod_impact'] = prod_meta_model.predict(all_prods_agg)
all_prods_agg = all_prods_agg.reset_index().merge(product_impact_scores, on='product_id', how='left')
all_prods_agg['prod_value_add_score'] = all_prods_agg['actual_prod_impact'] - all_prods_agg['predicted_prod_impact']
prod_final_scores = all_prods_agg[['product_id', 'predicted_prod_impact', 'prod_value_add_score']]

all_cats_agg = pd.crosstab(df_combined['category_id'], df_combined['event_type'])
all_cats_agg['predicted_cat_impact'] = cat_meta_model.predict(all_cats_agg)
all_cats_agg = all_cats_agg.reset_index().merge(category_impact_scores, on='category_id', how='left')
all_cats_agg['cat_value_add_score'] = all_cats_agg['actual_cat_impact'] - all_cats_agg['predicted_cat_impact']
cat_final_scores = all_cats_agg[['category_id', 'predicted_cat_impact', 'cat_value_add_score']]

# --- 3.2 v7'nin Tüm Kullanıcı Özelliklerini Hesaplama ---
print("v7'nin tüm kullanıcı seviyesi özellikleri hesaplanıyor...")
user_v7_features = df_combined.groupby('user_id').agg(
    user_total_events=('event_type', 'count'),
    user_unique_products_viewed=('product_id', 'nunique'),
    user_first_seen=('event_time', 'min'),
    user_last_seen=('event_time', 'max')
)
user_v7_features['user_lifespan_days'] = (user_v7_features['user_last_seen'] - user_v7_features['user_first_seen']).dt.days
user_buy_counts = df_combined[df_combined['event_type'] == 'BUY'].groupby('user_id').size().rename('user_buy_count')
user_v7_features = user_v7_features.join(user_buy_counts)
user_v7_features['user_purchase_rate'] = user_v7_features['user_buy_count'] / user_v7_features['user_total_events']
user_v7_features.drop(['user_first_seen', 'user_last_seen'], axis=1, inplace=True)

# --- 3.3 Ana Fonksiyon: v7 + v16 + v18 Sentezi ---
def create_final_features_v19(df, is_train=True):
    print(f"\n{'Train' if is_train else 'Test'} seti için v19 Sentez özellikleri oluşturuluyor...")
    df = df.sort_values(by=['user_session', 'event_time'])
    
    df['time_diff'] = df.groupby('user_session')['event_time'].diff().dt.total_seconds()
    df['event_order'] = df.groupby('user_session').cumcount() + 1
    
    agg_dict = {
        'user_id': 'first', 'event_type': 'count', 'product_id': ['nunique', list],
        'category_id': ['nunique', list], 'event_time': ['min', 'max', lambda x: x.dt.hour.mean(), lambda x: x.dt.dayofweek.mean()],
        'time_diff': ['mean', 'max', 'std', 'median'], 'event_order': 'mean'
    }
    if is_train: agg_dict['session_value'] = 'first'
    
    sf = df.groupby('user_session').agg(agg_dict)
    sf['user_session_original'] = df.groupby('user_session')['user_session_original'].first()

    col_names = [
        'user_id', 'event_count', 'unique_products', 'products', 'unique_categories', 'categories',
        'start_time', 'end_time', 'avg_hour', 'avg_dayofweek', 'avg_time_diff', 'max_time_diff',
        'std_time_diff', 'median_time_diff', 'avg_event_order',
    ]
    if is_train:
        col_names.append('session_value')
    
    col_names.append('user_session_original')

    sf.columns = col_names
    
    sf['duration'] = (sf['end_time'] - sf['start_time']).dt.total_seconds()
    sf['avg_event_order_pct'] = sf['avg_event_order'] / sf['event_count']
    
    event_counts = pd.crosstab(df['user_session'], df['event_type'])
    sf = sf.join(event_counts)
    for event in ['VIEW', 'ADD_CART', 'REMOVE_CART', 'BUY']:
        if event not in sf.columns: sf[event] = 0
    
    sf['net_cart_additions'] = sf['ADD_CART'] - sf['REMOVE_CART']
    sf['view_to_add_cart_rate'] = sf['ADD_CART'] / (sf['VIEW'] + 1e-6)
    sf['add_cart_to_buy_rate'] = sf['BUY'] / (sf['ADD_CART'] + 1e-6)
    sf['view_to_buy_rate'] = sf['BUY'] / (sf['VIEW'] + 1e-6)
    sf['did_purchase'] = (sf['BUY'] > 0).astype(int)

    sf = sf.merge(user_v7_features, on='user_id', how='left')
    sf = sf.merge(user_final_scores, on='user_id', how='left')
    
    # Ürün meta özelliklerini ekleyen bölüm geri eklendi.
    prod_scores_map = prod_final_scores.set_index('product_id')
    def get_prod_meta_scores(products):
        if not products:
            return pd.Series([0,0], index=['mean_pred_prod_impact', 'mean_prod_value_add'])
        scores = prod_scores_map.reindex(list(set(products)))
        if scores.empty:
            return pd.Series([0,0], index=['mean_pred_prod_impact', 'mean_prod_value_add'])
        mean_scores = scores[['predicted_prod_impact', 'prod_value_add_score']].mean()
        mean_scores = mean_scores.rename(index={'predicted_prod_impact': 'mean_pred_prod_impact', 'prod_value_add_score': 'mean_prod_value_add'})
        return mean_scores
        
    meta_prod_df = sf['products'].apply(get_prod_meta_scores)
    sf = pd.concat([sf, meta_prod_df], axis=1)

    cat_scores_map = cat_final_scores.set_index('category_id')
    def get_cat_meta_scores(categories):
        if not categories:
            return pd.Series([0,0], index=['mean_pred_cat_impact', 'mean_cat_value_add'])
        scores = cat_scores_map.reindex(list(set(categories)))
        if scores.empty:
            return pd.Series([0,0], index=['mean_pred_cat_impact', 'mean_cat_value_add'])
        mean_scores = scores[['predicted_cat_impact', 'cat_value_add_score']].mean()
        mean_scores = mean_scores.rename(index={'predicted_cat_impact': 'mean_pred_cat_impact', 'cat_value_add_score': 'mean_cat_value_add'})
        return mean_scores
        
    meta_cat_df = sf['categories'].apply(get_cat_meta_scores)
    sf = pd.concat([sf, meta_cat_df], axis=1)

    sf = sf.set_index('user_session_original', drop=True)
    
    sf.drop(['user_id', 'products', 'categories', 'start_time', 'end_time'], axis=1, inplace=True)
    return sf

df_session_train = create_final_features_v19(df_train_filtered, is_train=True)
df_session_test = create_final_features_v19(df_test_filtered, is_train=False)

train_cols = df_session_train.columns.drop('session_value')
df_session_test = df_session_test.reindex(columns=train_cols)
df_session_test.fillna(0, inplace=True)

df_session_train.to_csv(OUT_TRAIN_PATH, index=True)
df_session_test.to_csv(OUT_TEST_PATH, index=True)
print(f"\nİşlenmiş veriler kaydedildi. Train shape: {df_session_train.shape}, Test shape: {df_session_test.shape}")
print("\nBölüm 3 Tamamlandı.")
gc.collect()

# ==============================================================================
# --- BÖLÜM 4: NİHAİ ANA MODEL EĞİTİMİ (v19) ---
# ==============================================================================
print("\n" + "="*50)
print("Bölüm 4: Nihai Ana Model Eğitimi (v19) Başladı.")
print("="*50)

# --- Dosya Yolları ---
BASE_DIR = './'
IN_TRAIN_PATH = os.path.join(BASE_DIR, 'processed/train_processed_v19.csv')
IN_TEST_PATH = os.path.join(BASE_DIR, 'processed/test_processed_v19.csv')
OUT_MODEL_PATH = os.path.join(BASE_DIR, 'models/catboost_model_v19.cbm')
OUT_FEATURES_PATH = os.path.join(BASE_DIR, 'models/features_v19.json')

# --- İşlenmiş Veriyi Yükleme ---
try:
    df_train = pd.read_csv(IN_TRAIN_PATH, index_col=0)
except FileNotFoundError:
    print(f"HATA: '{IN_TRAIN_PATH}' dosyası bulunamadı. Lütfen ilk script'i çalıştırdığınızdan emin olun.")
    exit()
    
df_train.dropna(subset=['session_value'], inplace=True)
y = df_train['session_value']
X = df_train.drop('session_value', axis=1)
features = X.columns.tolist()
y_log = np.log1p(y)

# 80-20 train-test bölme
X_train, X_val, y_train_log, y_val_log = train_test_split(X, y_log, test_size=0.2, shuffle=True, random_state=42)
# Orijinal değerler de ayrı tutulacak MSE hesabı için
y_train_orig, y_val_orig = train_test_split(y, test_size=0.2, shuffle=True, random_state=42)

print(f"Nihai özellik sayısı: {len(features)}")
print(f"Eğitim seti boyutu: {X_train.shape[0]}, Doğrulama seti boyutu: {X_val.shape[0]}")

print("\nCatBoost Ana Modeli eğitimi başlıyor...")
# DEĞİŞİKLİK: Ana modelin parametreleri artırıldı.
model = CatBoostRegressor(
    iterations=10000,
    learning_rate=0.015,
    depth=8,
    l2_leaf_reg=5,
    loss_function='Tweedie:variance_power=1.8',
    eval_metric='RMSE',
    random_seed=42,
    verbose=500,
    early_stopping_rounds=400
)
model.fit(X_train, y_train_log, eval_set=(X_val, y_val_log))

val_preds = np.expm1(model.predict(X_val))
rmse = np.sqrt(mean_squared_error(np.expm1(y_val_log), val_preds))
mse = mean_squared_error(y_val_orig, val_preds)
print(f"\nValidation RMSE: {rmse:.4f}")
print(f"Validation MSE: {mse:.4f}")

print("\nTüm veri üzerinde nihai ana model eğitiliyor...")
final_model = CatBoostRegressor(**model.get_params())
final_model.set_params(iterations=model.get_best_iteration())
final_model.fit(X, y_log, verbose=False)

final_model.save_model(OUT_MODEL_PATH)
with open(OUT_FEATURES_PATH, 'w') as f:
    json.dump(features, f)

print(f"Model '{OUT_MODEL_PATH}' dosyasına kaydedildi.")
print(f"Özellik listesi '{OUT_FEATURES_PATH}' dosyasına kaydedildi.")
gc.collect()


# ==============================================================================
# --- BÖLÜM 5: SUBMISSION OLUŞTURMA (v19 - NİHAİ DÜZELTİLMİŞ) ---
# ==============================================================================
print("\n" + "="*50)
print("Bölüm 5: Submission Oluşturma (v19) Başladı.")
print("="*50)

# --- Gerekli Dosyaları Yükleme ---
IN_SUBMISSION_PATH = os.path.join(BASE_DIR, 'sample_submission.csv')
RAW_TRAIN_PATH = os.path.join(BASE_DIR, 'train.csv')
RAW_TEST_PATH = os.path.join(BASE_DIR, 'test.csv')
OUT_SUBMISSION_PATH = os.path.join(BASE_DIR, 'submissions/submission_v19.csv')

df_test_processed = pd.read_csv(IN_TEST_PATH, index_col=0)
df_submission_template = pd.read_csv(IN_SUBMISSION_PATH)
df_test_raw_leak = pd.read_csv(RAW_TEST_PATH)
df_test_raw_leak = fix_anomalous_sessions(df_test_raw_leak) # Ham test setine de original_session'ı ekle

# Modelin beklediği sırayı garantile
df_test_processed = df_test_processed[features]

# --- Tahminleri Yapma ---
print("Test seti üzerinde tahminler yapılıyor...")
predictions = np.expm1(final_model.predict(df_test_processed))
predictions[predictions < 0] = 0

# Tahminleri, İŞLENMİŞ (düzeltilmiş) session ID'leri ile bir DataFrame'e koy
df_predictions = pd.DataFrame({'user_session': df_test_processed.index, 'predicted_value': predictions})

# Orijinal session ID'lerini tahminlere ekle
session_map = df_test_raw_leak[['user_session', 'user_session_original']].drop_duplicates().set_index('user_session')
df_predictions = df_predictions.merge(session_map, on='user_session', how='left')


# --- Adım 1: Normal Tahminleri Eşleme ---
submission_map = df_predictions.set_index('user_session_original')['predicted_value'].to_dict()
df_submission = df_submission_template.copy()
df_submission['session_value'] = df_submission['user_session'].map(submission_map)


# --- Adım 2: Anormal Seanslar için Akıllı Doldurma ---
session_user_counts = pd.read_csv(RAW_TEST_PATH).groupby('user_session')['user_id'].nunique()
anormal_sessions_orig = session_user_counts[session_user_counts > 1].index.tolist()
print(f"\n{len(anormal_sessions_orig)} adet anormal seans için akıllı toplama işlemi yapılıyor...")

for session_id in anormal_sessions_orig:
    constituent_preds = df_predictions[df_predictions['user_session_original'] == session_id]
    
    if not constituent_preds.empty:
        total_value = constituent_preds['predicted_value'].sum()
        df_submission.loc[df_submission['user_session'] == session_id, 'session_value'] = total_value
        print(f"'{session_id}' için {len(constituent_preds)} parçanın tahmini toplandı: {total_value:.4f}")

# --- Adım 3: Nihai Adım: Doğrulanmış Sızıntıyı Ezme ---
print("\nDoğrulanmış sızıntı (verified leaks) uygulanıyor...")
df_train_leak = df_train_filtered  # Filtrelenmiş train verisini kullan
train_session_users = df_train_leak.groupby('user_session_original')['user_id'].apply(set)
test_session_users = df_test_filtered.groupby('user_session_original')['user_id'].apply(set)

common_sessions_final = set(train_session_users.index).intersection(set(test_session_users.index))
verified_leaked_sessions = {sid for sid in common_sessions_final if train_session_users.get(sid) == test_session_users.get(sid)}
leak_map = df_train_leak[df_train_leak['user_session_original'].isin(verified_leaked_sessions)].groupby('user_session_original')['session_value'].first().to_dict()
print(f"{len(leak_map)} adet tahmin, doğrulanmış gerçek değerlerle eziliyor...")

# update metodu için index ayarlama
df_submission = df_submission.set_index('user_session')
df_submission['session_value'].update(pd.Series(leak_map))
df_submission.reset_index(inplace=True)

expected_rows = 30789
print(f"\nNihai dosyadaki satır sayısı: {len(df_submission)}. Beklenen: {expected_rows}")
if len(df_submission) != expected_rows:
    print(f"UYARI: Satır sayısı eşleşmiyor! Lütfen kontrol edin.")
else:
    print("Satır sayısı DOĞRU.")

# Olası NaN değerleri 0 ile doldur
df_submission['session_value'].fillna(0, inplace=True)

df_submission.to_csv(OUT_SUBMISSION_PATH, index=False)
print("\n" + "="*50)
print(f"İŞLEM TAMAMLANDI! Nihai submission dosyası '{OUT_SUBMISSION_PATH}' oluşturuldu.")
print("="*50)
print("Nihai Dosyanın Başı:")
print(df_submission.head())

# ==============================================================================
# --- BÖLÜM 6: ID ETKİLERİ ANALİZİ ---
# ==============================================================================
print("\n" + "="*80)
print("BÖLÜM 6: ÜRÜN ID, KATEGORİ ID VE KULLANICI ID ETKİLERİ ANALİZİ")
print("="*80)

# Analiz sonuçlarını saklamak için liste
analysis_results = []

# İşlenmiş train verisini yükle
df_train_analysis = pd.read_csv(IN_TRAIN_PATH, index_col=0)
df_train_analysis.dropna(subset=['session_value'], inplace=True)
y_analysis = df_train_analysis['session_value']
y_log_analysis = np.log1p(y_analysis)

# ID ile ilgili özellikleri belirle
user_features = [col for col in df_train_analysis.columns if 'user' in col.lower()]
product_features = [col for col in df_train_analysis.columns if 'prod' in col.lower()]
category_features = [col for col in df_train_analysis.columns if 'cat' in col.lower()]

# Çakışan özellikleri temizle
user_specific_features = [col for col in user_features if 'prod' not in col.lower() and 'cat' not in col.lower()]
product_specific_features = [col for col in product_features if 'user' not in col.lower() and 'cat' not in col.lower()]
category_specific_features = [col for col in category_features if 'user' not in col.lower() and 'prod' not in col.lower()]

# Genel özellikler (multiple ID türlerinde kullanılanlar)
general_features = ['unique_products', 'unique_categories']  # Bu özellikler hem ürün hem kategori ile ilgili

# Tüm ID özelliklerini birleştir (tekrar olmayacak şekilde)
all_id_features = list(set(user_specific_features + product_specific_features + category_specific_features + general_features))

print(f"Kullanıcı özellikleri ({len(user_specific_features)}): {user_specific_features}")
print(f"Ürün özellikleri ({len(product_specific_features)}): {product_specific_features}")  
print(f"Kategori özellikleri ({len(category_specific_features)}): {category_specific_features}")
print(f"Genel özellikler ({len(general_features)}): {general_features}")

# Baseline özellikler (ID'ler hariç)
baseline_features = [col for col in df_train_analysis.columns if col not in all_id_features and col != 'session_value']
print(f"Baseline özellikler ({len(baseline_features)}): {baseline_features[:5]}...")

def train_and_evaluate_model(X_features, feature_name, iteration_count=2000):
    """Model eğitip MSE hesaplayan fonksiyon"""
    print(f"\n{feature_name} ile model eğitiliyor...")
    
    X_temp = df_train_analysis[X_features]
    X_train_temp, X_val_temp, y_train_temp, y_val_temp = train_test_split(
        X_temp, y_log_analysis, test_size=0.2, random_state=42
    )
    y_train_orig_temp, y_val_orig_temp = train_test_split(
        y_analysis, test_size=0.2, random_state=42
    )
    
    model_temp = CatBoostRegressor(
        iterations=iteration_count,
        learning_rate=0.02,
        depth=6,
        verbose=0,
        random_seed=42
    )
    model_temp.fit(X_train_temp, y_train_temp)
    
    val_preds_temp = np.expm1(model_temp.predict(X_val_temp))
    mse_temp = mean_squared_error(y_val_orig_temp, val_preds_temp)
    rmse_temp = np.sqrt(mse_temp)
    
    print(f"{feature_name} - MSE: {mse_temp:.4f}, RMSE: {rmse_temp:.4f}")
    return mse_temp, rmse_temp

# 1. Baseline Model (ID'ler olmadan)
print("\n" + "-"*60)
print("1. BASELINE MODEL (ID ÖZELLİKLERİ OLMADAN)")
print("-"*60)
baseline_mse, baseline_rmse = train_and_evaluate_model(baseline_features, "Baseline")
analysis_results.append({
    'Model': 'Baseline (ID\'ler olmadan)',
    'Özellik Sayısı': len(baseline_features),
    'MSE': baseline_mse,
    'RMSE': baseline_rmse,
    'MSE İyileştirme': 0,
    'MSE İyileştirme %': 0
})

# 2. Baseline + Sadece Ürün ID
print("\n" + "-"*60)
print("2. BASELINE + SADECE ÜRÜN ID ÖZELLİKLERİ")
print("-"*60)
# unique_products'ı ürün özelliği olarak kabul et, tekrar ekleme
product_features_combined = baseline_features + product_specific_features
# unique_products zaten product_specific_features'da değilse ekle
if 'unique_products' not in product_features_combined:
    product_features_combined.append('unique_products')
product_mse, product_rmse = train_and_evaluate_model(product_features_combined, "Baseline + Ürün ID")
product_improvement = baseline_mse - product_mse
product_improvement_pct = (product_improvement / baseline_mse) * 100
analysis_results.append({
    'Model': 'Baseline + Ürün ID',
    'Özellik Sayısı': len(product_features_combined),
    'MSE': product_mse,
    'RMSE': product_rmse,
    'MSE İyileştirme': product_improvement,
    'MSE İyileştirme %': product_improvement_pct
})

# 3. Baseline + Sadece Kullanıcı ID
print("\n" + "-"*60)
print("3. BASELINE + SADECE KULLANICI ID ÖZELLİKLERİ")
print("-"*60)
user_features_combined = baseline_features + user_specific_features
user_mse, user_rmse = train_and_evaluate_model(user_features_combined, "Baseline + Kullanıcı ID")
user_improvement = baseline_mse - user_mse
user_improvement_pct = (user_improvement / baseline_mse) * 100
analysis_results.append({
    'Model': 'Baseline + Kullanıcı ID',
    'Özellik Sayısı': len(user_features_combined),
    'MSE': user_mse,
    'RMSE': user_rmse,
    'MSE İyileştirme': user_improvement,
    'MSE İyileştirme %': user_improvement_pct
})

# 4. Baseline + Sadece Kategori ID
print("\n" + "-"*60)
print("4. BASELINE + SADECE KATEGORİ ID ÖZELLİKLERİ")
print("-"*60)
category_features_combined = baseline_features + category_specific_features
# unique_categories'ı kategori özelliği olarak kabul et, tekrar ekleme
if 'unique_categories' not in category_features_combined:
    category_features_combined.append('unique_categories')
category_mse, category_rmse = train_and_evaluate_model(category_features_combined, "Baseline + Kategori ID")
category_improvement = baseline_mse - category_mse
category_improvement_pct = (category_improvement / baseline_mse) * 100
analysis_results.append({
    'Model': 'Baseline + Kategori ID',
    'Özellik Sayısı': len(category_features_combined),
    'MSE': category_mse,
    'RMSE': category_rmse,
    'MSE İyileştirme': category_improvement,
    'MSE İyileştirme %': category_improvement_pct
})

# 5. Tüm özellikler birlikte
print("\n" + "-"*60)
print("5. TÜM ÖZELLİKLER BİRLİKTE (BASELINE + TÜM ID'LER)")
print("-"*60)
# Tüm benzersiz özellikleri birleştir
all_features_combined = list(set(baseline_features + user_specific_features + product_specific_features + category_specific_features + ['unique_products', 'unique_categories']))
all_mse, all_rmse = train_and_evaluate_model(all_features_combined, "Tüm Özellikler")
all_improvement = baseline_mse - all_mse
all_improvement_pct = (all_improvement / baseline_mse) * 100
analysis_results.append({
    'Model': 'Tüm Özellikler Birlikte',
    'Özellik Sayısı': len(all_features_combined),
    'MSE': all_mse,
    'RMSE': all_rmse,
    'MSE İyileştirme': all_improvement,
    'MSE İyileştirme %': all_improvement_pct
})

# Analiz sonuçları tablosu
print("\n" + "="*100)
print("ID ETKİLERİ ANALİZ TABLOSU")
print("="*100)

results_df = pd.DataFrame(analysis_results)
print(results_df.to_string(index=False, float_format='%.4f'))

# En iyi performansları göster
print("\n" + "="*100)
print("ÖNEMLI BULGULAR")
print("="*100)

best_individual_model = results_df.iloc[1:4].loc[results_df.iloc[1:4]['MSE İyileştirme %'].idxmax()]
print(f"• En iyi tekil ID etkisi: {best_individual_model['Model']} - %{best_individual_model['MSE İyileştirme %']:.2f} iyileştirme")

worst_individual_model = results_df.iloc[1:4].loc[results_df.iloc[1:4]['MSE İyileştirme %'].idxmin()]
print(f"• En zayıf tekil ID etkisi: {worst_individual_model['Model']} - %{worst_individual_model['MSE İyileştirme %']:.2f} iyileştirme")

total_improvement = results_df.iloc[-1]['MSE İyileştirme %']
sum_individual_improvements = results_df.iloc[1:4]['MSE İyileştirme %'].sum()
synergy_effect = total_improvement - sum_individual_improvements

print(f"• Bireysel ID etkilerinin toplamı: %{sum_individual_improvements:.2f}")
print(f"• Tüm ID'lerin birlikte etkisi: %{total_improvement:.2f}")
print(f"• Sinerji etkisi: %{synergy_effect:.2f}")

if synergy_effect > 0:
    print("  → ID'ler birlikte kullanıldığında POZİTİF sinerji var!")
else:
    print("  → ID'ler birlikte kullanıldığında NEGATİF etkileşim var!")

print("\n" + "="*100)